In [12]:
import psycopg2
import pandas as pd

# Database connection details
username = 'test'
password = 'test123'
hostname = '192.168.1.141'
port = '5432'
database_name = 'mydb'

# Establish connection
conn = psycopg2.connect(
    dbname=database_name,
    user=username,
    password=password,
    host=hostname,
    port=port
)

# SQL query
sql_query = "SELECT  TO_TIMESTAMP(lasttradetime) AT TIME ZONE 'Asia/Kolkata' AS lasttradetime, open,high,low,close,openinterest FROM whole_indices_futures WHERE instrumentidentifier = 'NIFTY10APR2422700CE' order by lasttradetime;"

# Execute SQL query
cursor = conn.cursor()
cursor.execute(sql_query)

# Fetch all the results
rows = cursor.fetchall()

# Convert to DataFrame
df = pd.DataFrame(rows, columns=[desc[0] for desc in cursor.description])

# Close cursor and connection
cursor.close()
conn.close()

# Display the DataFrame
print(df)


            lasttradetime    open    high     low   close  openinterest
0     2024-03-13 11:13:00  212.50  212.50  212.50  212.50             0
1     2024-03-13 11:13:00  212.50  212.50  212.50  212.50             0
2     2024-03-13 11:14:00  212.50  212.50  212.50  212.50             0
3     2024-03-13 11:14:00  212.50  212.50  212.50  212.50             0
4     2024-03-13 11:15:00  212.50  212.50  212.50  212.50             0
...                   ...     ...     ...     ...     ...           ...
13017 2024-04-09 15:28:00   41.85   42.80   41.80   41.80       8715150
13018 2024-04-09 15:29:00   41.85   41.95   40.00   40.00       8715150
13019 2024-04-09 15:29:00   41.85   41.95   40.00   40.00       8715150
13020 2024-04-09 15:30:00   40.00   40.00   40.00   40.00       8715150
13021 2024-04-09 15:30:00   40.00   40.00   40.00   40.00       8715150

[13022 rows x 6 columns]


In [13]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler

# Assuming df is your DataFrame
df.set_index('lasttradetime', inplace=True)
df.fillna(method="ffill", inplace=True)  

scaler = MinMaxScaler()
df_scaled = scaler.fit_transform(df)

lookback_window = 3
features = 5

def create_sequences(data, lookback_window):
    X, y = [], []
    for i in range(len(data) - lookback_window - 1): 
        X.append(data[i:(i + lookback_window), :])
        y.append(data[i + lookback_window, 0:5]) 
    return np.array(X), np.array(y)

model = Sequential()
model.add(LSTM(units=100, return_sequences=True, input_shape=(lookback_window, features)))
model.add(LSTM(units=100))
model.add(Dense(units=5)) 
model.compile(loss='mean_squared_error', optimizer='adam')

# Create sequences for LSTM
X, y = create_sequences(df_scaled, lookback_window)

# Train the model using all data
model.fit(X, y, epochs=50, batch_size=32)

# Make predictions
y_pred = model.predict(X)
y_pred_original = scaler.inverse_transform(y_pred)
data = pd.DataFrame(y_pred_original, columns=['open', 'high', 'low' ,'close','tradedqty'])
data


Epoch 1/50


C:\Users\santh\AppData\Local\Temp\ipykernel_15052\3523302334.py:9: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method="ffill", inplace=True)
c:\Users\santh\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


407/407 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - loss: 0.0108
Epoch 2/50
407/407 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 1.0572e-04
Epoch 3/50
407/407 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 1.1022e-04
Epoch 4/50
407/407 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 9.8965e-05
Epoch 5/50
407/407 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 1.1576e-04
Epoch 6/50
407/407 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 9.1397e-05
Epoch 7/50
407/407 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 1.5495e-04
Epoch 8/50
407/407 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 8.3124e-05
Epoch 9/50
407/407 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 9.1732e-05
Epoch 10/50
407/407 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 9.6891e-05
Epoch 11/50
407/407 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 8.5648e-05
Epoch 12/50
407/407 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 6.8965e-05
Epoch 13/50
407/407 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 8.4681e-05
Epoch 14/50
407/407 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 8.2298e-05
Epoch 15/50
407/

,open,high,low,close,tradedqty
0,213.602737,213.471695,213.401108,212.902817,1.265200e+05
1,213.602737,213.471695,213.401108,212.902817,1.265200e+05
2,213.602737,213.471695,213.401108,212.902817,1.265200e+05
3,213.602737,213.471695,213.401108,212.902817,1.265200e+05
4,213.602737,213.471695,213.401108,212.902817,1.265200e+05
...,...,...,...,...,...
13013,41.597954,42.254311,40.871357,41.378204,9.293847e+06
13014,41.232212,41.801674,40.799797,41.013218,9.098665e+06
13015,41.281429,42.074306,40.902390,41.325375,8.903630e+06
13016,41.312420,41.358856,39.564884,39.588093,8.706146e+06


In [14]:
def generate_predictions(model, last_sequence, num_predictions):
    predictions = []
    current_sequence = last_sequence.reshape(1, lookback_window, features)  # Reshape to match model input shape
    for _ in range(num_predictions):
        next_prediction = model.predict(current_sequence)  # Predict the next data point
        predictions.append(next_prediction[0])  # Append the prediction to the list of predictions
        current_sequence = np.append(current_sequence[:, 1:, :], next_prediction.reshape(1, 1, features), axis=1)  # Update the current sequence
    return predictions

# Generate predictions for the next 60 data points
last_sequence = X_test[-1]  # Using the last sequence from the test data
num_predictions = 720
predicted_data = generate_predictions(model, last_sequence, num_predictions)

# Inverse transform the predictions to obtain the original scale
predicted_data_original = scaler.inverse_transform(predicted_data)

# Create a DataFrame with the predicted values and column names
predicted_df = pd.DataFrame(predicted_data_original, columns=['open', 'high', 'low', 'close', 'volume'])

# Print the predicted data
print(predicted_df)



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━

In [15]:
frequency = 'T'
start_time = '2024-04-09 15:30:00'
end_time = pd.to_datetime(start_time) + pd.DateOffset(minutes=len(predicted_df) - 1)
date_range = pd.date_range(start=start_time, end=end_time, freq=frequency)
predicted_df['lasttradetime'] = date_range
predicted_df.set_index('lasttradetime', inplace=True)
predicted_df.to_csv('NIFTY10APR2422700CE.csv')


predicted_df

C:\Users\santh\AppData\Local\Temp\ipykernel_15052\3351771881.py:4: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  date_range = pd.date_range(start=start_time, end=end_time, freq=frequency)


,open,high,low,close,volume
lasttradetime,,,,,
2024-04-09 15:30:00,39.798024,40.532780,38.956587,39.595015,8.712248e+06
2024-04-09 15:31:00,39.294468,39.926127,38.654535,39.123819,8.710966e+06
2024-04-09 15:32:00,38.954275,39.610206,38.317139,38.793208,8.710958e+06
2024-04-09 15:33:00,38.573628,39.144001,37.945076,38.318759,8.704973e+06
2024-04-09 15:34:00,38.207675,38.687993,37.555310,37.841927,8.701116e+06
...,...,...,...,...,...
2024-04-10 03:25:00,-110.585812,-111.124869,-102.381114,-106.974499,1.473240e+06
2024-04-10 03:26:00,-110.585823,-111.124869,-102.381125,-106.974488,1.473240e+06
2024-04-10 03:27:00,-110.585812,-111.124858,-102.381136,-106.974488,1.473240e+06


In [10]:
# Assuming predicted_df is your DataFrame
description = predicted_df.describe()

# Access the "min" and "max" rows
low_value = description.loc['min']
high_value = description.loc['max']

print("Low value:")
print(low_value)
print("\nHigh value:")
print(high_value)


Low value:
open     -3.402597e-01
high     -4.058388e+00
low       3.491534e+00
close    -2.780225e-01
volume    1.827660e+06
Name: min, dtype: float64

High value:
open      6.569061e+01
high      6.713445e+01
low       6.343421e+01
close     6.518500e+01
volume    7.734547e+06
Name: max, dtype: float64
